In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import json
import psycopg2


with open('config.json') as f:
    conf = json.load(f)

In [6]:
#Trying the database route
conn_str = "host={} dbname={} user={} password={}".format(conf['host'],conf['database'], conf['user'], conf['passw'])
conn = psycopg2.connect(conn_str)

In [7]:
plays = pd.read_sql('select * from play_player ', con=conn)
games = pd.read_sql('select * from game ', con=conn)
players = pd.read_sql('select * from player ', con=conn)

In [8]:
#dropping non-grouping columns + defensive fields. Might create a defensive scoring dataframe later.
def_cols = []
for d in plays.columns.values.tolist():
    if d.startswith('defense'):
        def_cols.append(d)

def_cols.extend(('drive_id','play_id'))
plays_dropped = plays.drop(columns=def_cols)


In [9]:
#Aggregating stats to game-player level
plays_sum = plays_dropped.groupby(by=['gsis_id','player_id','team']).sum().reset_index()

In [40]:
#fixing inconsistent complete of name columns
def fix_names(row):
    """This function looks up the chain of various name columns to fix null column issues"""
    if not row['gsis_name']:
        if not row['full_name']:
            if not row['last_name']:
                    return None
            else:
                if not row['first_name']:
                    return row['first_name'][0]+'.'+row['last_name']
                else:
                    return row['last_name']
        else:
            return row['full_name']
    else:
        return row['gsis_name']

In [51]:
#Applying fix_names function from above to create a new column
players.loc[:,'name_fixed'] = players.apply(fix_names,axis=1)

In [52]:
#dropping extra name columns and other unneccessary columbs now
players_dropped = players[['name_fixed','player_id','position','years_pro','height','weight']]

In [53]:
#joining players to aggregated plays
pp = plays_sum.merge(players_dropped, on='player_id', how='left')

In [54]:
#dropping unnecessary columns from games
games_dropped = games[['gsis_id','start_time','week','day_of_week','season_year','season_type','home_team','away_team','home_score','away_score']]

In [55]:
#joining players-plays to games
ppg = pp.merge(games_dropped, on='gsis_id', how='left')

In [60]:
ppg.to_csv('/Users/ianbury/player-game-stats.csv')